In [ ]:
from google.colab import drive
drive.mount('/root/gdrive')


Mounted at /root/gdrive


In [ ]:
!pip install transformers
!pip install tqdm
!pip install nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.9 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score

import numpy as np

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, SequentialSampler


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
pip install openprompt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00


In [ ]:
cd /root/gdrive/MyDrive/OpenPrompt

/root/gdrive/MyDrive/OpenPrompt


In [ ]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 8.7 MB/s eta 0:00:00


In [ ]:
from openprompt.data_utils.text_classification_dataset import AgnewsProcessor  # load specific news processor
from openprompt.data_utils.data_sampler import FewShotSampler # few shot
from openprompt.plms import load_plm
from openprompt.prompts import ManualTemplate
from openprompt.prompts import ManualVerbalizer
from openprompt import PromptDataLoader
from openprompt.prompts import SoftVerbalizer
from openprompt import PromptForClassification
import torch
from transformers import  AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
#print(dataset)
from openprompt.data_utils.text_classification_dataset import ToxicityProcessor


/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [ ]:
dataset = {}
# trainvalid_dataset = processor.get_train_examples(dataset_path)
dataset['train'] = ToxicityProcessor().get_train_examples("/root/gdrive/MyDrive/toxicity_data")
#sampler  = FewShotSampler(num_examples_per_label=3, num_examples_per_label_dev=3, also_sample_dev=True)
#dataset['train'], dataset['validation'] = sampler(dataset['train'])
dataset['test'] = ToxicityProcessor().get_test_examples("/root/gdrive/MyDrive/toxicity_data")
print(len(dataset['train']))
print(dataset['train'][0])


51182
{
  "guid": "83aa72a87f749fdc",
  "label": 1,
  "meta": {},
  "text_a": "lennie injured remainder season gettin prono three fucked brown crooked cant get game alnwick never get game spur fucked lawrence club going ahhahahahah fuck porno three hate scum hope mother saw knocking bint bunch filthy pikey bastard",
  "text_b": "",
  "tgt_text": null
}



In [ ]:
classes = ['0','1']

In [ ]:
plm, tokenizer, model_config, WrapperClass=load_plm("bert", "bert-base-cased") # load pretrained model
#plm, tokenizer, model_config, WrapperClass = load_plm("t5", "t5-base")
#plm, tokenizer, model_config, WrapperClass=load_plm("opt", "facebook/opt-1.3b")


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:


# define the template
template = ManualTemplate(tokenizer=tokenizer, text= '{"placeholder":"text_a"} It was {"mask"}',)
#template = ManualTemplate(tokenizer=tokenizer, text= 'What is the toxicity label of {"placeholder":"text_a"}? It was {"mask"}',)
#template = ManualTemplate(tokenizer=tokenizer, text= 'Analyzing the comment {"placeholder":"text_a"}, it can be determined that the toxicity label is {"mask"}',)
verbalizer = ManualVerbalizer(
    classes = classes,
    label_words = {
        #"0": ['non-toxic'],
        "0": ['non-toxic'],
        "1": ['toxic'],
    },
    tokenizer = tokenizer,
)

#verbalizer = SoftVerbalizer(tokenizer, plm, num_classes=2)

wrapped_example = template.wrap_one_example(dataset['train'][0])
#print('wrapped_example:',wrapped_example)


In [ ]:
# in the pipline base
# maximum 512 tokens for bert, tail truncation, not fill the mask with target text #2048
train_dataloader = PromptDataLoader(dataset=dataset['train'], template=template, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=512, batch_size=8, teacher_forcing=False, predict_eos_token=False,
    truncate_method='tail')

#val_dataloader = PromptDataLoader(dataset=dataset['validation'], template=template, tokenizer=tokenizer,
#    tokenizer_wrapper_class=WrapperClass, max_seq_length=512, batch_size=2, teacher_forcing=False, predict_eos_token=False,
#    truncate_method='tail')

test_dataloader = PromptDataLoader(dataset=dataset['test'], template=template, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=512, batch_size=8, teacher_forcing=False, predict_eos_token=False,
    truncate_method='tail')


tokenizing: 335it [00:00, 639.46it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (656 > 512). Running this sequence through the model will result in indexing errors
tokenizing: 51182it [01:18, 652.13it/s]
tokenizing: 12796it [00:17, 713.49it/s]


In [ ]:
use_cuda = True
# build the prompt model
prompt_model = PromptForClassification(plm=plm,template=template, verbalizer=verbalizer, freeze_plm=False)
if use_cuda:
    prompt_model = prompt_model.cuda()


In [ ]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

prompt_model.eval()

# the predictions and true labels
preds_accumulator = []
true_labels_accumulator = []

with torch.no_grad():
    for inputs in tqdm(test_dataloader, desc="Predicting", unit="batch"):
        if use_cuda:
            inputs = {k: v.to(device) for k, v in inputs.items() if torch.is_tensor(v)}
        logits = prompt_model(inputs)
        labels = inputs['label']
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        labels = labels.cpu().numpy()
        # accumulate
        preds_accumulator.extend(preds)
        true_labels_accumulator.extend(labels)


accuracy = accuracy_score(true_labels_accumulator, preds_accumulator)
print('Zero-shot Accuracy:', accuracy)
# other metrics
precision = precision_score(true_labels_accumulator, preds_accumulator, average='weighted')  # Considering class imbalance
recall = recall_score(true_labels_accumulator, preds_accumulator, average='weighted')  # Considering class imbalance
f1 = f1_score(true_labels_accumulator, preds_accumulator, average='weighted')  # Considering class imbalance
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

report = classification_report(true_labels_accumulator, preds_accumulator)
print('Classification Report:')
print(report)


Predicting: 100%|██████████| 1600/1600 [1:30:56<00:00,  3.41s/batch]

Zero-shot Accuracy: 0.9002813379180994
Precision: 0.8562632184614402
Recall: 0.9002813379180994
F1 Score: 0.8624433184781369
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.99      0.95     11547
           1       0.40      0.04      0.08      1249

    accuracy                           0.90     12796
   macro avg       0.65      0.52      0.51     12796
weighted avg       0.86      0.90      0.86     12796



In [ ]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

prompt_model.eval()

# the predictions and true labels
preds_accumulator = []
true_labels_accumulator = []

with torch.no_grad():
    for inputs in tqdm(test_dataloader, desc="Predicting", unit="batch"):
        if use_cuda:
            inputs = {k: v.to(device) for k, v in inputs.items() if torch.is_tensor(v)}
        logits = prompt_model(inputs)
        labels = inputs['label']
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        labels = labels.cpu().numpy()
        # accumulate
        preds_accumulator.extend(preds)
        true_labels_accumulator.extend(labels)


accuracy = accuracy_score(true_labels_accumulator, preds_accumulator)
print('Zero-shot Accuracy:', accuracy)
# other metrics
precision = precision_score(true_labels_accumulator, preds_accumulator, average='weighted')  # Considering class imbalance
recall = recall_score(true_labels_accumulator, preds_accumulator, average='weighted')  # Considering class imbalance
f1 = f1_score(true_labels_accumulator, preds_accumulator, average='weighted')  # Considering class imbalance
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

report = classification_report(true_labels_accumulator, preds_accumulator)
print('Classification Report:')
print(report)


Predicting: 100%|██████████| 1600/1600 [02:28<00:00, 10.75batch/s]

Zero-shot Accuracy: 0.7207721162863395
Precision: 0.8144562028663024
Recall: 0.7207721162863395
F1 Score: 0.7627420111822506
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.78      0.83     11547
           1       0.07      0.15      0.10      1249

    accuracy                           0.72     12796
   macro avg       0.48      0.47      0.47     12796
weighted avg       0.81      0.72      0.76     12796



In [ ]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

prompt_model.eval()

# the predictions and true labels
preds_accumulator = []
true_labels_accumulator = []

with torch.no_grad():
    for inputs in tqdm(test_dataloader, desc="Predicting", unit="batch"):
        if use_cuda:
            inputs = {k: v.to(device) for k, v in inputs.items() if torch.is_tensor(v)}
        logits = prompt_model(inputs)
        labels = inputs['label']
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        labels = labels.cpu().numpy()
        # accumulate
        preds_accumulator.extend(preds)
        true_labels_accumulator.extend(labels)


accuracy = accuracy_score(true_labels_accumulator, preds_accumulator)
print('Zero-shot Accuracy:', accuracy)
# other metrics
precision = precision_score(true_labels_accumulator, preds_accumulator, average='weighted')  # Considering class imbalance
recall = recall_score(true_labels_accumulator, preds_accumulator, average='weighted')  # Considering class imbalance
f1 = f1_score(true_labels_accumulator, preds_accumulator, average='weighted')  # Considering class imbalance
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

report = classification_report(true_labels_accumulator, preds_accumulator, digits=3)
print('Classification Report:')
print(report)


Predicting: 100%|██████████| 1600/1600 [02:29<00:00, 10.73batch/s]

Zero-shot Accuracy: 0.7207721162863395
Precision: 0.8144562028663024
Recall: 0.7207721162863395
F1 Score: 0.7627420111822506
Classification Report:
              precision    recall  f1-score   support

           0      0.895     0.782     0.835     11547
           1      0.070     0.151     0.096      1249

    accuracy                          0.721     12796
   macro avg      0.482     0.467     0.465     12796
weighted avg      0.814     0.721     0.763     12796

